# Scraping Google Classroom Part 2
**This file is made for scraping GC's assigment in one class and showing whether the assigment has filled or not.**

In [1]:
# Import Selenium and Time
from selenium import webdriver
import time

In [2]:
# Open Web Driver and get to url
Driver = webdriver.Chrome()
Driver.get("https://classroom.google.com/u/0/h")
time.sleep(10)

In [ ]:
# Ask user to enter their email and password. This will be used for login to google classroom automatically
Email = input("Please input your email: ").strip()
Password = input("Please input your password").strip()

In [5]:
# Login automatically using data that has been stored
Driver.find_element_by_id("identifierId").send_keys(Email)
Driver.find_element_by_css_selector("div.VfPpkd-RLmnJb").click()
time.sleep(5)
Driver.find_element_by_css_selector("input.whsOnd.zHQkBf").send_keys(Password)
Driver.find_element_by_css_selector("div.VfPpkd-RLmnJb").click()
time.sleep(3)

In [6]:
# Web Scraper arrived at home! Choose Class/Subject that you want to scrap. This function will return index of class
def search_class():
    while True:
        i = 0
        Classes = Driver.find_elements_by_css_selector("div.Tc9hUd")
        for Class in Classes:
            print("["+str(i)+"] "+Class.text.split("\n")[0])
            i = i + 1

        print("["+str(len(Classes))+"] "+"All Class")
        Class_chosen = int(input("Please Class's number: ").strip())

        if (Class_chosen >= 0 and Class_chosen <= len(Classes)):
            return i

In [9]:
def goback(how_many_times):
    for i in range(0, how_many_times):
        Driver.back()
        time.sleep(10)

def assignment_tab():
    tabs = Driver.find_elements_by_css_selector("div.wZTANe")
    return (tabs[1].find_element_by_tag_name("a")).get_attribute("href")

def scroll_down(n):
    for i in range(0, n):
        time.sleep(4)
        Driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        try:
            Button = Driver.find_element_by_css_selector("div.uArJ5e.UQuaGc.kCyAyd.nZ34k")
            Button.click()
        except:
            break
        time.sleep(4)
        
def scraping_assignment():
    list_of_assignments = []
    
    Assignments = Driver.find_elements_by_class_name("jHgkif.raZwr")
    
    for Assignment in Assignments:
        Assignments1 = Assignment.find_elements_by_css_selector('li[guidedhelpid="classworkTopicListGh"]')
        for Assign in Assignments1:
            Assign.click()
            time.sleep(1)
            data = Assign.text.split("\n")
            date = Assign.find_element_by_css_selector("div.wCDkmf.T3FW5d.YVvGBb.dDKhVc").text.split("Tenggat: ")
            status = ''
            try:
                status = Assign.find_element_by_css_selector("span.u7S8tc.YVvGBb").text
            except:
                continue
            assignment_id = Assign.get_attribute("data-dom-id").split("|")
            list_of_assignments.append({
                "assignment name" : data[1],
                "assignment id" : assignment_id[0],
                "deadlines" : date[len(date)-1],
                "status" : [data[0], status]
            })

    return list_of_assignments

def scroll_up():
    Driver.execute_script('window.scrollTo(0, document.head.scrollHeight);')
    
def scraping_subject():
    
    # Subject's info (1)
    subject_name = Driver.find_element_by_id("UGb2Qe").text
    
    # Goto Assignment tab
    tab = assignment_tab()
    Driver.get(tab)
    
    time.sleep(5)
    scroll_down(6) # Scroll 6 times in the Assignment tab
    scroll_up() 
    
    # Subject's Info (2)
    assignments = scraping_assignment()
    
    # Goto Member tab
#     tab = member_tab()
#     Driver.get(tab)
    
#     time.sleep(5)
#     scroll(6) # Scroll 6 times in the Member tab
    
    #Subject's Info (3)
#     member = scraping_member()
    
    return {
        'subject name' : subject_name,
        'assignments' : assignments#,
#         'members' : member
    }  

In [11]:
classes = Driver.find_elements_by_css_selector("div.Tc9hUd")
list_of_subjects = []

class_index = search_class()

if (class_index == len(classes)):
    for i in range(0, len(classes)):
        # Entering Class/Subject
        classes = Driver.find_elements_by_css_selector("div.Tc9hUd")
        classes[i].click()

        # Scraping Subject
        time.sleep(5)
        list_of_subjects.append(scraping_subject())

        goback(2)
else:
    classes = Driver.find_elements_by_css_selector("div.Tc9hUd")
    classes[class_index].click()
    
    # Scraping Subject
    time.sleep(5)
    list_of_subjects.append(scraping_subject())
    
    goback(2)

[0] Komunikasi Teknis dan Bisnis
[1] TekPro D4 A 2020
[2] Proyek 1 D4 2019/2020
[3] Teknik Pemrograman 1A
[4] Konsep TIK IA
[5] All Class


Please Class's number:  5


In [12]:
list_of_subjects

[{'subject name': 'Komunikasi Teknis dan Bisnis',
  'assignments': [{'assignment name': 'Penilaian Antar Teman',
    'assignment id': '117498421948',
    'deadlines': '17 Jul 23.59',
    'status': ['Selesai Tugas', 'Diserahkan']},
   {'assignment name': 'Presensi Perkuliahan',
    'assignment id': '92728317520',
    'deadlines': 'Tidak ada batas waktu',
    'status': ['Selesai Tugas', 'Diserahkan']},
   {'assignment name': 'Pembuatan Proposal Perancangan Aplikasi',
    'assignment id': '94190145942',
    'deadlines': '6 Mei',
    'status': ['Selesai Tugas', 'Diserahkan']},
   {'assignment name': 'Mengenal Dokumentasi Bisnis pada proyek Pengembangan Perangkat Lunak',
    'assignment id': '89173588405',
    'deadlines': '22 Apr 16.00',
    'status': ['Selesai Tugas', 'Diserahkan']},
   {'assignment name': 'Pengumpulan Revisi Laporan dan Poster',
    'assignment id': '117328227507',
    'deadlines': '16 Jul 15.00',
    'status': ['Selesai Tugas', 'Diserahkan']}]},
 {'subject name': 'TekPr